# FRB Scattering Analysis

**Production notebook for MCMC scattering analysis**

Workflow:
1. Load configuration and create pipeline
2. (Optional) Refine initial guess interactively
3. Run MCMC fitting
4. Generate diagnostic plots and corner plot

## 0. Imports and Setup

In [ ]:
import os
import sys
from pathlib import Path
import yaml
import numpy as np

# Add paths
sys.path.insert(0, str(Path.cwd().parent))
sys.path.insert(0, str(Path.cwd().parent.parent))

from scat_analysis.burstfit_pipeline import BurstPipeline
from scat_analysis.burstfit_interactive import InitialGuessWidget

print('[OK] Imports successful')

## 1. Configuration

**Select your burst configuration file:**

In [ ]:
# Configuration file path
config_file = '../configs/bursts/dsa/casey_dsa.yaml'

# Load configuration
with open(config_file, 'r') as f:
    config = yaml.safe_load(f)

# Load telescope configuration
telescope_cfg = config.get('telescope_cfg', '../configs/telescopes.yaml')
with open(telescope_cfg, 'r') as f:
    tel_config = yaml.safe_load(f)

print(f'[OK] Loaded config: {config_file}')
print(f'    Burst: {config.get("burst_name", "unknown")}')
print(f'    Telescope: {config.get("telescope", "unknown")}')

## 2. Create Pipeline

In [ ]:
# Create pipeline instance
pipeline_params = {
    'telescope_cfg': telescope_cfg,
    'burst_cfg': config_file,
    'output_dir': config.get('output_dir', './output'),
}

pipe = BurstPipeline(**pipeline_params)
print('[OK] Pipeline created')

## 3. Interactive Initial Guess Refinement (Optional)

Use the widget to visually refine initial parameters before MCMC.

In [ ]:
# Create interactive widget
guess_widget = InitialGuessWidget(
    data=pipe.data,
    time_axis=pipe.time_axis,
    freq_axis=pipe.freq_axis,
    initial_params=pipe.initial_params,
    param_bounds=pipe.param_bounds,
)

# Display widget (in Jupyter)
guess_widget.display()

In [ ]:
# Apply refined initial guess to pipeline
refined_params = guess_widget.get_params()
pipe.seed_single = refined_params
print(f'[OK] Applied refined initial guess')
print(f'    chi2 = {guess_widget.chi2:.2f}')

## 4. Run MCMC Fitting

In [ ]:
# Run full pipeline
results = pipe.run_full(
    model_scan=False,   # Skip model scan, use single model
    save_chains=True,   # Save MCMC chains
    save_plots=True,    # Save diagnostic plots
)

print('[OK] MCMC fitting complete')

## 5. Post-Analysis

In [ ]:
# Extract results
sampler = results.get('sampler')
best_params = results.get('best_params')
param_names = results.get('param_names')

# Check convergence
if sampler is not None:
    try:
        tau = sampler.get_autocorr_time(quiet=True)
        print(f'Autocorrelation times: {tau}')
    except Exception as e:
        print(f'[~] Could not compute autocorrelation: {e}')

# Print best-fit parameters
print('\nBest-fit parameters:')
for name, val in zip(param_names, best_params):
    print(f'  {name}: {val:.4f}')

In [ ]:
# Generate corner plot
from scat_analysis.burstfit_corner import make_corner_plot

fig = make_corner_plot(
    sampler=sampler,
    param_names=param_names,
    truths=best_params,
)

# Save
output_dir = Path(config.get('output_dir', './output'))
fig.savefig(output_dir / 'corner_plot.png', dpi=150, bbox_inches='tight')
print(f'[OK] Corner plot saved to {output_dir}/corner_plot.png')

## Analysis Complete

### Output Files
- Diagnostic plots: `{output_dir}/*.png`
- MCMC chains: `{output_dir}/chains.npy`
- Corner plot: `{output_dir}/corner_plot.png`